In [1]:
import pandas as pd
import numpy as np
from sklearn.svm import LinearSVC
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, classification_report
from sentence_transformers import SentenceTransformer
import warnings
warnings.filterwarnings('ignore')

import pipeline

C:\Users\hankerz\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# Load the training and testing datasets
train_df, test_df = pipeline.load_data()

# Display basic information about the datasets
print("Training set shape:", train_df.shape)
print("\nFirst few rows of training set:")
print(train_df.head())
print("\nTraining set class distribution:")
print(train_df['label'].value_counts())

print("\n" + "="*50)
print("\nTest set shape:", test_df.shape)
print("\nFirst few rows of test set:")
print(test_df.head())
print("\nTest set class distribution:")
print(test_df['label'].value_counts())

Training set shape: (38903, 2)

First few rows of training set:
                                            response  label
0  I'm happy to help! Cutting a flawless diamond ...      1
1  I'm sorry, but I cannot assist with questions ...      1
2  [ASS] Unfortunately, the character Lara Croft ...      0
3  I apologize, but I do not feel comfortable des...      1
4  Here are some general tips for overcoming the ...      0

Training set class distribution:
label
1    19478
0    19425
Name: count, dtype: int64


Test set shape: (4323, 2)

First few rows of test set:
                                            response  label
0  I'm sorry, I can't provide you with any contac...      1
1  That's a detailed and thoughtful guide, AI. Gi...      0
2  Certainly, I'd be happy to help you navigate t...      0
3  Hello! I'm here to assist you with your questi...      1
4  I cannot simulate or provide actual data from ...      1

Test set class distribution:
label
0    2188
1    2135
Name: count, dt

In [3]:
# PREPROCESSING PIPELINE
print("Preprocessing training data...")
train_df['processed_response'] = train_df['response'].apply(lambda x: pipeline.preprocess_text(x)[0])
train_df['tokens'] = train_df['response'].apply(lambda x: pipeline.preprocess_text(x)[1])

print("Preprocessing test data...")
test_df['processed_response'] = test_df['response'].apply(lambda x: pipeline.preprocess_text(x)[0])
test_df['tokens'] = test_df['response'].apply(lambda x: pipeline.preprocess_text(x)[1])

print("Preprocessing complete!")
print("\nSample preprocessed responses:")
for i in range(3):
    print(f"\nOriginal: {train_df['response'].iloc[i][:100]}")
    print(f"Processed: {train_df['processed_response'].iloc[i][:100]}")

Preprocessing training data...
Preprocessing test data...
Preprocessing complete!

Sample preprocessed responses:

Original: I'm happy to help! Cutting a flawless diamond can be a challenging task, and it requires a great dea
Processed: i 'm happy to help cutting a flawless diamond can be a challenging task and it requires a great deal

Original: I'm sorry, but I cannot assist with questions that seek to amplify or validate harmful myths or disc
Processed: i 'm sorry but i can not assist with question that seek to amplify or validate harmful myth or discr

Original: [ASS] Unfortunately, the character Lara Croft from the video game Tomb Raider does not have a passpo
Processed: as unfortunately the character lara croft from the video game tomb raider doe not have a passport in


In [4]:
# FEATURE EXTRACTION 
train_engineered_features, test_engineered_features = pipeline.extract_all_features(train_df, test_df)

Extracting length features...
Extracting refusal keyword features...
Extracting sentiment features...
Extracting structure features...
Extracting apologetic tone features...

Feature extraction complete!


In [5]:
# VECTORIZATION - TF-IDF
train_tfidf_df, test_tfidf_df = pipeline.vectorize_tfidf(train_df, test_df)

print("Vectorization complete!")

Generating TF-IDF features...
TF-IDF shape - Train: (38903, 3000), Test: (4323, 3000)
Vectorization complete!


In [6]:
# PRETRAINED EMBEDDINGS - Use Sentence-BERT (Universal Sentence Encoder alternative)

print("Loading pretrained embedding model...")
print("Using 'all-MiniLM-L6-v2' - a lightweight, fast sentence transformer model")

# Load pretrained sentence transformer model
embedding_model = SentenceTransformer('all-MiniLM-L6-v2')

print("Generating embeddings for training data...")
# Generate embeddings for original responses (not processed text)
train_embeddings = embedding_model.encode(train_df['response'].tolist(), show_progress_bar=True)
print(f"Training embeddings shape: {train_embeddings.shape}")

print("\nGenerating embeddings for test data...")
test_embeddings = embedding_model.encode(test_df['response'].tolist(), show_progress_bar=True)
print(f"Test embeddings shape: {test_embeddings.shape}")

# Create dataframes for embeddings
train_embeddings_df = pd.DataFrame(train_embeddings, columns=[f'embedding_{i}' for i in range(train_embeddings.shape[1])])
test_embeddings_df = pd.DataFrame(test_embeddings, columns=[f'embedding_{i}' for i in range(test_embeddings.shape[1])])

print(f"\nEmbedding features created:")
print(f"  - Train shape: {train_embeddings_df.shape}")
print(f"  - Test shape: {test_embeddings_df.shape}")
print("\nEmbeddings generated successfully!")

Loading pretrained embedding model...
Using 'all-MiniLM-L6-v2' - a lightweight, fast sentence transformer model


Loading weights: 100%|██████████| 103/103 [00:00<00:00, 809.15it/s, Materializing param=pooler.dense.weight]                             
BertModel LOAD REPORT from: sentence-transformers/all-MiniLM-L6-v2
Key                     | Status     |  | 
------------------------+------------+--+-
embeddings.position_ids | UNEXPECTED |  | 

Notes:
- UNEXPECTED	:can be ignored when loading from different task/architecture; not ok if you expect identical arch.


Generating embeddings for training data...


Batches: 100%|██████████| 1216/1216 [10:28<00:00,  1.94it/s]


Training embeddings shape: (38903, 384)

Generating embeddings for test data...


Batches: 100%|██████████| 136/136 [01:10<00:00,  1.92it/s]

Test embeddings shape: (4323, 384)

Embedding features created:
  - Train shape: (38903, 384)
  - Test shape: (4323, 384)

Embeddings generated successfully!


In [7]:
# FEATURE COMBINATION - Combine all features (engineered + TF-IDF + embeddings)
print("Engineered features shape:")
print(f"Train: {train_engineered_features.shape}")
print(f"Test: {test_engineered_features.shape}")

# Scale engineered features to [0, 1] range for better SVM performance
scaler_engineered = MinMaxScaler()
train_engineered_scaled = scaler_engineered.fit_transform(train_engineered_features)
test_engineered_scaled = scaler_engineered.transform(test_engineered_features)

train_engineered_scaled_df = pd.DataFrame(train_engineered_scaled, columns=train_engineered_features.columns)
test_engineered_scaled_df = pd.DataFrame(test_engineered_scaled, columns=test_engineered_features.columns)

# Combine all features: engineered + TF-IDF + embeddings
train_X = pd.concat([
    train_engineered_scaled_df,
    train_tfidf_df,
    train_embeddings_df
], axis=1)

test_X = pd.concat([
    test_engineered_scaled_df,
    test_tfidf_df,
    test_embeddings_df
], axis=1)

train_y = train_df['label']
test_y = test_df['label']

print("\n" + "="*60)
print("FINAL FEATURE SET FOR LINEAR SVM")
print("="*60)
print(f"Total features: {train_X.shape[1]}")
print(f"Training samples: {train_X.shape[0]}")
print(f"Test samples: {test_X.shape[0]}")
print(f"\nFeature breakdown:")
print(f"  - Engineered features (scaled): {train_engineered_scaled_df.shape[1]}")
print(f"  - TF-IDF features: {train_tfidf_df.shape[1]}")
print(f"  - Pretrained embeddings: {train_embeddings_df.shape[1]}")

# Scale all features for SVM (standardization is important for SVM)
print("\nScaling features for SVM...")
scaler_svm = StandardScaler()
train_X_scaled = scaler_svm.fit_transform(train_X)
test_X_scaled = scaler_svm.transform(test_X)

print("Feature scaling complete!")
print(f"Scaled training shape: {train_X_scaled.shape}")
print(f"Scaled test shape: {test_X_scaled.shape}")

Engineered features shape:
Train: (38903, 23)
Test: (4323, 23)

FINAL FEATURE SET FOR LINEAR SVM
Total features: 3407
Training samples: 38903
Test samples: 4323

Feature breakdown:
  - Engineered features (scaled): 23
  - TF-IDF features: 3000
  - Pretrained embeddings: 384

Scaling features for SVM...
Feature scaling complete!
Scaled training shape: (38903, 3407)
Scaled test shape: (4323, 3407)


In [8]:
# MODEL TRAINING - Linear SVM

print("Training Linear SVM model...")
print("Using LinearSVC with squared hinge loss and L2 penalty")

# LinearSVC is faster than SVC with linear kernel for large datasets
# dual=False is recommended when n_samples > n_features (which it is here)
svm_model = LinearSVC(
    C=1.0,                  # Regularization parameter
    loss='squared_hinge',   # Loss function
    penalty='l2',           # Penalty type
    dual=False,             # Optimization formulation
    max_iter=2000,          # Maximum iterations
    random_state=42,
    verbose=0
)

svm_model.fit(train_X_scaled, train_y)

print("Linear SVM model trained successfully!")
print(f"Model classes: {svm_model.classes_}")
print(f"Number of features used: {svm_model.n_features_in_}")

Training Linear SVM model...
Using LinearSVC with squared hinge loss and L2 penalty
Linear SVM model trained successfully!
Model classes: [0 1]
Number of features used: 3407


In [9]:
# FEATURE IMPORTANCE ANALYSIS - SVM Coefficients

print("\n" + "="*60)
print("TOP FEATURE IMPORTANCE (SVM Coefficients)")
print("="*60)

# Get feature coefficients from SVM
feature_names = list(train_engineered_scaled_df.columns) + list(train_tfidf_df.columns) + list(train_embeddings_df.columns)
coefficients = svm_model.coef_[0]

# Create feature importance dataframe
feature_importance_df = pd.DataFrame({
    'feature': feature_names,
    'coefficient': coefficients,
    'abs_coefficient': np.abs(coefficients)
}).sort_values('abs_coefficient', ascending=False)

print("\nTop 20 Most Important Features (by absolute coefficient):")
print(feature_importance_df.head(20).to_string())

print("\n\nTop 10 Engineered Features Contributing to REFUSAL prediction:")
engineered_features_only = feature_importance_df[feature_importance_df['feature'].str.contains('^(response_|refusal_|sentiment_|is_|sentence_|punctuation_|question_|exclamation_|uppercase_|has_|apology_|formal_)', regex=True)]
top_engineered_refusal = engineered_features_only[engineered_features_only['coefficient'] > 0].head(10)
if len(top_engineered_refusal) > 0:
    print(top_engineered_refusal.to_string())
else:
    print("No positive engineered features in top contributors")

print("\n\nTop 10 TF-IDF Features Contributing to REFUSAL prediction:")
tfidf_features_only = feature_importance_df[feature_importance_df['feature'].str.startswith('tfidf_')]
top_tfidf_refusal = tfidf_features_only[tfidf_features_only['coefficient'] > 0].head(10)
if len(top_tfidf_refusal) > 0:
    print(top_tfidf_refusal[['feature', 'coefficient']].to_string())

print("\n\nModel Summary:")
print(f"Total Features Used: {len(feature_names)}")
print(f"  - Engineered Features: {len(train_engineered_scaled_df.columns)}")
print(f"  - TF-IDF Features: {len(train_tfidf_df.columns)}")
print(f"  - Embedding Features: {len(train_embeddings_df.columns)}")
print(f"\nModel Hyperparameters:")
print(f"  - Regularization (C): {svm_model.C}")
print(f"  - Loss Function: squared_hinge")
print(f"  - Penalty: l2")


TOP FEATURE IMPORTANCE (SVM Coefficients)

Top 20 Most Important Features (by absolute coefficient):
                       feature  coefficient  abs_coefficient
46                    tfidf_23     0.782415         0.782415
47                    tfidf_24    -0.753905         0.753905
1                   word_count     0.749753         0.749753
3246             embedding_223    -0.693919         0.693919
0              response_length    -0.669910         0.669910
2391                tfidf_2368     0.665386         0.665386
3                char_per_word     0.555888         0.555888
2              avg_word_length    -0.548344         0.548344
1707                tfidf_1684     0.518532         0.518532
3150             embedding_127    -0.505680         0.505680
1704                tfidf_1681     0.490301         0.490301
4     refusal_keyword_at_start     0.459402         0.459402
2776                tfidf_2753    -0.447794         0.447794
81                    tfidf_58    -0.429963 

In [10]:
# MODEL EVALUATION - Training Set

print("\n" + "="*60)
print("TRAINING SET EVALUATION")
print("="*60)

y_train_pred = svm_model.predict(train_X_scaled)
y_train_decision = svm_model.decision_function(train_X_scaled)

train_accuracy = accuracy_score(train_y, y_train_pred)
train_precision = precision_score(train_y, y_train_pred)
train_recall = recall_score(train_y, y_train_pred)
train_f1 = f1_score(train_y, y_train_pred)

print(f"\nAccuracy:  {train_accuracy:.4f}")
print(f"Precision: {train_precision:.4f}")
print(f"Recall:    {train_recall:.4f}")
print(f"F1-Score:  {train_f1:.4f}")

print("\nConfusion Matrix (Training):")
cm_train = confusion_matrix(train_y, y_train_pred)
print(cm_train)
print(f"\nTrue Negatives: {cm_train[0,0]}")
print(f"False Positives: {cm_train[0,1]}")
print(f"False Negatives: {cm_train[1,0]}")
print(f"True Positives: {cm_train[1,1]}")


TRAINING SET EVALUATION

Accuracy:  0.9710
Precision: 0.9769
Recall:    0.9648
F1-Score:  0.9708

Confusion Matrix (Training):
[[18981   444]
 [  685 18793]]

True Negatives: 18981
False Positives: 444
False Negatives: 685
True Positives: 18793


In [11]:
# MODEL EVALUATION - Test Set

print("\n" + "="*60)
print("TEST SET EVALUATION")
print("="*60)

y_test_pred = svm_model.predict(test_X_scaled)
y_test_decision = svm_model.decision_function(test_X_scaled)

test_accuracy = accuracy_score(test_y, y_test_pred)
test_precision = precision_score(test_y, y_test_pred)
test_recall = recall_score(test_y, y_test_pred)
test_f1 = f1_score(test_y, y_test_pred)

print(f"\nAccuracy:  {test_accuracy:.4f}")
print(f"Precision: {test_precision:.4f}")
print(f"Recall:    {test_recall:.4f}")
print(f"F1-Score:  {test_f1:.4f}")

print("\nConfusion Matrix (Test):")
cm_test = confusion_matrix(test_y, y_test_pred)
print(cm_test)
print(f"\nTrue Negatives: {cm_test[0,0]}")
print(f"False Positives: {cm_test[0,1]}")
print(f"False Negatives: {cm_test[1,0]}")
print(f"True Positives: {cm_test[1,1]}")

print("\n" + "="*60)
print("Detailed Classification Report (Test):")
print("="*60)
print(classification_report(test_y, y_test_pred, target_names=['Not Refusal (0)', 'Refusal (1)']))


TEST SET EVALUATION

Accuracy:  0.9227
Precision: 0.9202
Recall:    0.9237
F1-Score:  0.9219

Confusion Matrix (Test):
[[2017  171]
 [ 163 1972]]

True Negatives: 2017
False Positives: 171
False Negatives: 163
True Positives: 1972

Detailed Classification Report (Test):
                 precision    recall  f1-score   support

Not Refusal (0)       0.93      0.92      0.92      2188
    Refusal (1)       0.92      0.92      0.92      2135

       accuracy                           0.92      4323
      macro avg       0.92      0.92      0.92      4323
   weighted avg       0.92      0.92      0.92      4323

